In [1]:
import pandas as pd
import numpy as np
import datetime
import math
import os
import matplotlib.pyplot as plt
from pylab import rcParams

plt.style.use('ggplot')
%pylab inline --no-import-all

Populating the interactive namespace from numpy and matplotlib


In [2]:
pwd()

'/home/yuichiro/Miniconda/Weather/GribToCSV'

In [3]:
popt = np.array([ 0.00220261, -0.00783915,  0.00496154])
rho0 = 1.225

def power_curve_func(x,cutin,popt):
    return np.where(x > 14, 0.3,np.where(x < cutin, 0, popt[0]*x**2+popt[1]*x+popt[2]))

def wind_turbine_output(wind_values,temp_values,MSL_values,Pr,V_rat, V_CIN, V_CO):   
    # Pr:rated power, V_rat: rated wind speed,V_CIN: cut-in speed, V=CO: cut-out speed,rho: air dencity
    rho = MSL_values / (2.87*273.15+2.87*temp_values)
    Pwt=(np.where((V_rat<=wind_values) & (wind_values <= V_CO), Pr,\
                  np.where((V_CIN<=wind_values) & (wind_values <= V_rat), \
                           Pr*(wind_values-V_CIN)/(V_rat-V_CIN),0 )))
    return Pwt,rho

# Set or Modify parameters below!!

In [4]:
# インプットデータ
year = "2019"
month = "06"
day = "16"
endday = "24"
# Check time value
time = "000000"


while int(day) < int(endday):

    day2 = str(int(day)+2)
    day3 = str(int(day)+3)

    if int(day) < 10:
        day = "0"+str(int(day))

    if int(day2) < 10:
        day2 = "0"+str(int(day2))

    if int(day3) < 10:
        day3 = "0"+str(int(day3))

    # 開始日本時間（UTC+9）
    # time=000000 09:00-12:00
    start_TimeJP = year+'-'+month+'-'+day+' 10:00'
    finish_TimeJP = year+'-'+month+'-'+day2+' 13:00'

    # time=120000 21:00-00:00
    #start_TimeJP =year+'-'+month+'-'+day+' 22:00'
    #finish_TimeJP =year+'-'+month+'-'+day3+' 01:00'

    file_dir_path = "Output_CSV/"+year+"/"+month+"/"+day

    # UTC time to Japan in time:000000
    TimeJP = pd.date_range(start_TimeJP, finish_TimeJP, freq='H')
    TimeJP.columns = ['Datetime in Japan']
    Date = pd.DataFrame({'DateTime': TimeJP.values})
    # Date
    Date['DateTime'] = pd.to_datetime(Date['DateTime'])
    # Date.dtypes
    if len(Date) == 52:
        print("OK")
    else:
        print("Error")
        print(len(Date))

    Date['Date'] = Date['DateTime']
    Date['Year'] = Date['DateTime'] .dt.year
    Date['Month'] = Date['DateTime'] .dt.month
    Date['Day'] = Date['DateTime'] .dt.day
    Date['Hour'] = Date['DateTime'] .dt.hour

    input_tempFH_0015 = pd.read_csv(
        file_dir_path+'/FH_00-15_temperature'+year+'_'+month+'_'+day+'_'+time+'.csv')
    input_tempFH_1633 = pd.read_csv(
        file_dir_path+'/FH_16-33_temperature'+year+'_'+month+'_'+day+'_'+time+'.csv')
    input_tempFH_3439 = pd.read_csv(
        file_dir_path+'/FH_34-39_temperature'+year+'_'+month+'_'+day+'_'+time+'.csv')
    input_tempFH_4051 = pd.read_csv(
        file_dir_path+'/FH_40-51_temperature'+year+'_'+month+'_'+day+'_'+time+'.csv')

    input_temp = pd.concat([input_tempFH_0015, input_tempFH_1633, input_tempFH_3439,
                            input_tempFH_4051], axis=1).T.drop(["index", "lons", "lats"])
    input_temp.columns = ['temperature']
    input_temp = input_temp.reset_index()
    # input_temp

    input_UwindFH_0015 = pd.read_csv(
        file_dir_path+'/FH_00-15_Uwind'+year+'_'+month+'_'+day+'_'+time+'.csv')
    input_UwindFH_1633 = pd.read_csv(
        file_dir_path+'/FH_16-33_Uwind'+year+'_'+month+'_'+day+'_'+time+'.csv')
    input_UwindFH_3439 = pd.read_csv(
        file_dir_path+'/FH_34-39_Uwind'+year+'_'+month+'_'+day+'_'+time+'.csv')
    input_UwindFH_4051 = pd.read_csv(
        file_dir_path+'/FH_40-51_Uwind'+year+'_'+month+'_'+day+'_'+time+'.csv')

    input_Uwind = pd.concat([input_UwindFH_0015, input_UwindFH_1633, input_UwindFH_3439,
                             input_UwindFH_4051], axis=1).T.drop(["index", "lons", "lats"]).apply(abs)
    input_Uwind.columns = ['Uwind']

    input_Uwind = input_Uwind.reset_index()

    input_VwindFH_0015 = pd.read_csv(
        file_dir_path+'/FH_00-15_Vwind'+year+'_'+month+'_'+day+'_'+time+'.csv')
    input_VwindFH_1633 = pd.read_csv(
        file_dir_path+'/FH_16-33_Vwind'+year+'_'+month+'_'+day+'_'+time+'.csv')
    input_VwindFH_3439 = pd.read_csv(
        file_dir_path+'/FH_34-39_Vwind'+year+'_'+month+'_'+day+'_'+time+'.csv')
    input_VwindFH_4051 = pd.read_csv(
        file_dir_path+'/FH_40-51_Vwind'+year+'_'+month+'_'+day+'_'+time+'.csv')

    input_Vwind = pd.concat([input_VwindFH_0015, input_VwindFH_1633, input_VwindFH_3439,
                             input_VwindFH_4051], axis=1).T.drop(["index", "lons", "lats"]).apply(abs)
    input_Vwind.columns = ['Vwind']
    input_Vwind = input_Vwind.reset_index()

    input_radFH_0015 = pd.read_csv(
        file_dir_path+'/FH_00-15_radiation'+year+'_'+month+'_'+day+'_'+time+'.csv')
    input_radFH_1633 = pd.read_csv(
        file_dir_path+'/FH_16-33_radiation'+year+'_'+month+'_'+day+'_'+time+'.csv')
    input_radFH_3439 = pd.read_csv(
        file_dir_path+'/FH_34-39_radiation'+year+'_'+month+'_'+day+'_'+time+'.csv')
    input_radFH_4051 = pd.read_csv(
        file_dir_path+'/FH_40-51_radiation'+year+'_'+month+'_'+day+'_'+time+'.csv')

    input_rad = pd.concat([input_radFH_0015, input_radFH_1633, input_radFH_3439,
                           input_radFH_4051], axis=1).T.drop(["index", "lons", "lats"])
    input_rad.columns = ['radiation']
    input_rad = input_rad.reset_index()

    input_MSLFH_0015 = pd.read_csv(
        file_dir_path+'/FH_00-15_MSL'+year+'_'+month+'_'+day+'_'+time+'.csv')
    input_MSLFH_1633 = pd.read_csv(
        file_dir_path+'/FH_16-33_MSL'+year+'_'+month+'_'+day+'_'+time+'.csv')
    input_MSLFH_3439 = pd.read_csv(
        file_dir_path+'/FH_34-39_MSL'+year+'_'+month+'_'+day+'_'+time+'.csv')
    input_MSLFH_4051 = pd.read_csv(
        file_dir_path+'/FH_40-51_MSL'+year+'_'+month+'_'+day+'_'+time+'.csv')

    input_MSL = pd.concat([input_MSLFH_0015, input_MSLFH_1633, input_MSLFH_3439,
                           input_MSLFH_4051], axis=1).T.drop(["index", "lons", "lats"])
    input_MSL.columns = ['MSL']
    input_MSL = input_MSL.reset_index()

    # calc wind
    wind10 = np.square(input_Uwind['Uwind'].values) + \
        np.square(input_Vwind['Vwind'].values)
    wind10 = np.sqrt(wind10)
    wind10 = pd.DataFrame({'wind': wind10})
    wind10.columns = ['wind 10m[m/s]']

    forecast = pd.concat([Date, wind10, input_temp, input_rad,
                          input_Uwind, input_Vwind, input_MSL], axis=1, sort=False)

    # def wind_turbine_output(wind_values,temp_values,MSL_values,Pr,V_rat, V_CIN, V_CO)
    Pwt_values, rho_values = wind_turbine_output(
        wind10.values, forecast["temperature"].values, forecast["MSL"].values, 0.3, 20, 3, 100)

    # def power_curve_func(x,cutin,popt)
    Pwt_values_from_power_curve = power_curve_func(wind10.values, 3, popt)

    forecast['wind output STC[kW]'] = pd.Series(Pwt_values.reshape(-1))
    forecast['rho'] = pd.Series(rho_values.reshape(-1))
    forecast['wind output[kW]'] = forecast['wind output STC[kW]'] * \
        (forecast['rho']/rho0)
    forecast['wind output from power curve[kW]'] = pd.Series(
        Pwt_values_from_power_curve.reshape(-1))

    print(len(forecast))
    # forecast=forecast.set_index('DateTime')

    # 太陽光発電初期値

    # インプットデータ
    # input_rad = pd.read_csv('rad.csv')#encordingはUTF-8なので注意 unitは、MJ/m2
    input_csv_parameter = pd.read_csv('PV_input_parameter.csv')

    # 1列目以外を抽出
    input_csv_parameter = input_csv_parameter.iloc[:, [
        1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]]

    # float(1.367)#the solar constant [1.367 kW/m2]
    Gsc = float(input_csv_parameter.iat[0, 0])
    # the solar radiation at which the NOCT is defined [0.8 kW/m2]
    Gt_NOCT = float(input_csv_parameter.iat[0, 1])
    # the ambient temperature at which the NOCT is defined [20°C]
    Ta_NOCT = float(input_csv_parameter.iat[0, 2])+273.15
    # the nominal operating cell temperature [°C]
    Tc_NOCT = float(input_csv_parameter.iat[0, 3])+273.15
    # the PV cell temperature under standard test conditions [25°C]
    Tc_STC = float(input_csv_parameter.iat[0, 4])+273.15

    # the rated capacity of the PV array, meaning its power output under standard test conditions [kW]
    YPV = float(input_csv_parameter.iat[0, 6])
    ap = float(input_csv_parameter.iat[0, 7])  # Module Efficiency(Pmax)[%/℃]
    beta = float(input_csv_parameter.iat[0, 8])  # the slope of the surface [°]
    # the solar absorptance and the solar transmittance[%]
    eta_c = float(input_csv_parameter.iat[0, 9])
    fpv = float(input_csv_parameter.iat[0, 10])  # the PV derating factor [%]
    # the azimuth of the surface [°]
    ganma = float(input_csv_parameter.iat[0, 11])
    lam = float(input_csv_parameter.iat[0, 12])  # the longitude [°]
    phi = float(input_csv_parameter.iat[0, 13])  # the latitude [°]
    # the ground reflectance, which is also called the albedo [%]
    rho_g = float(input_csv_parameter.iat[0, 14])
    # the solar absorptance and the solar transmittance[%]
    tau_a = float(input_csv_parameter.iat[0, 15])
    # the incident radiation at standard test conditions [1 kW/m2]
    _Gt_STC = float(1)
    UL = (tau_a*Gt_NOCT)/(Tc_NOCT-Ta_NOCT)
    #UL = float(input_csv_parameter.iat[0,5])
    total_cost = float(input_csv_parameter.iat[0, 16])  # total cost
    Op_year = float(input_csv_parameter.iat[0, 17])  # oparation year
    cost = total_cost/(Op_year*365*24)  # cost[yen/kW/h]

    # ωの計算
    forecast['ts'] = forecast['Hour']
    forecast['ω'] = (forecast['ts'].values - 12)*15  # degree
    forecast['n'] = Date['DateTime'] .dt.dayofyear

    forecast['δ the solar declination'] = 23.45 * \
        np.sin(np.deg2rad(360*(284+forecast['n'].values)/365))  # degree

    delta = forecast['δ the solar declination'].values  # degree

    # 角度をdeg2rad
    omega = np.deg2rad(forecast['ω'].values)  # rad
    delta = np.deg2rad(delta)  # rad
    phi = math.radians(phi)  # rad
    beta = math.radians(beta)  # rad
    ganma = math.radians(ganma)  # rad
    lam = math.radians(lam)  # rad

    # cosθの計算
    forecast['cosθ'] = np.sin(delta)*math.sin(phi)*math.cos(beta)\
        - np.sin(delta)*math.cos(phi)*math.sin(beta)*math.cos(ganma)\
        + np.cos(delta)*math.cos(phi)*math.cos(beta)*np.cos(omega)\
        + np.cos(delta)*math.sin(phi)*math.sin(beta)*math.cos(ganma)*np.cos(omega)\
        + np.cos(delta)*math.sin(beta)*math.sin(ganma)*np.sin(omega)

    cos_theta = forecast['cosθ'].values

    forecast['θ(radian)'] = np.arccos(forecast['cosθ'].values)  # rad

    forecast['θ(degree)'] = np.rad2deg(forecast['θ(radian)'].values)  # degree

    Theta_deg = forecast['θ(radian)'].values  # rad

    # cosθzの計算
    forecast['cosθz'] = math.cos(phi)*np.cos(delta)*np.cos(omega)\
        + math.sin(phi)*np.sin(delta)

    cos_theta_z = forecast['cosθz'].values

    forecast['θz(radian)'] = np.arccos(forecast['cosθz'].values)  # rad

    forecast['θz(degree)'] = np.rad2deg(
        forecast['θz(radian)'].values)  # degree

    Theta_z_deg = forecast['θz(radian)'].values  # rad

    # theta degree ndarray
    theta = forecast['θ(radian)'].values  # rad

    # theta degree ndarray
    theta_z = forecast['θz(radian)'].values  # rad

    # decimals=1で小数点第2位を四捨五入する
    cos_theta = np.round(cos_theta, decimals=1)
    cos_theta_z = np.round(cos_theta_z, decimals=1)
    Theta_deg = np.round(Theta_deg, decimals=0)
    Theta_deg_z = np.round(Theta_z_deg, decimals=0)

    cos_theta = np.cos(Theta_deg)
    cos_theta_z = np.cos(Theta_z_deg)

    # Gon(the extraterrestrial normal radiation)[kW/m2]の計算
    forecast['Gon(the extraterrestrial normal radiation)[kW/m2]'] \
        = Gsc*(1+0.033*np.cos(np.deg2rad(360*forecast['n'].values/365)))  # np.deg2rad

    Gon = forecast['Gon(the extraterrestrial normal radiation)[kW/m2]'].values

    # Go(the extraterrestrial horizontal radiation [kW/m2])の計算
    forecast['Go(the extraterrestrial horizontal radiation)[kW/m2]'] \
        = Gon*np.abs(cos_theta_z)

    Go = forecast['Go(the extraterrestrial horizontal radiation)[kW/m2]'] .values

    # _Go(the extraterrestrial horizontal radiation averaged over the time step )[kW/m2]の計算

    def step_minus(x):  # 再帰関数　引数[2]-引数[1] ndarrayで返す ndarrayを引数
        x = list(x)
        y = [0]
        y.extend(x)  # 引数2
        x.extend([0])  # 引数1
        y = np.array(y)
        x = np.array(x)
        ans = y - x
        return ans[1:]  # [1:]or [:8760]

    forecast['_Go(the extraterrestrial horizontal radiation averaged over the time step )[kW/m2]'] \
        = (12/np.pi)*Gon*np.abs(np.cos(phi)*np.cos(delta)*step_minus(np.sin(omega))
                                + ((np.pi/180)*step_minus(omega)*np.sin(phi)*np.sin(delta)))

    _Go = forecast['_Go(the extraterrestrial horizontal radiation averaged over the time step )[kW/m2]']

    forecast['the global horizontal radiation[kW/m2]'] =\
        forecast['radiation'].values/1000

    _G = forecast['the global horizontal radiation[kW/m2]'].values
    forecast['the global horizontal radiation[kW/m2]'] = _G

    # ktの計算
    forecast['kt(the clearness index)'] = _G/_Go  # _G2
    kt = np.abs(forecast['kt(the clearness index)'].values)

    _Go = kt/_G

    # _Gdの計算
    # ２の３乗#累乗np.power(2, 3)
    def if_kt(x):
        x = np.abs(x)
        ans = np.where(x >= 0.8, 0.165, np.where(
            x < 0.22, 1-0.09*x, 0.95511-0.1604*x+4.388*x*x-16.638*x*x*x+12.336*x*x*x*x))
        return ans

    forecast['_Gd(the diffuse radiation)[kW/m2]'] = _G*if_kt(kt)
    _Gd = forecast['_Gd(the diffuse radiation)[kW/m2]'].values

    # _Gb(the beam radiation)の計算
    forecast['_Gb(the beam radiation)[kW/m2]'] = _G-_Gd

    _Gb = forecast['_Gb(the beam radiation)[kW/m2]'].values

    # _G
    forecast['_G'] = _Gd+_Gb

    # Rbの計算
    #input_data['Rb']= cos_theta/cos_theta_z
    forecast['Rb'] = np.abs(forecast['cosθ'].values/forecast['cosθz'].values)

    # thetaの計算がjupyterだとうまくいかない泣
    # input_data['Rb']=Rb
    Rb = forecast['Rb'].values

    # 発散の処理 要考察 yを超えるとzになる　
    def div(x, y, z):
        x = abs(x)
        ans = np.where(x > y, z, x)
        return ans
    #Rb = div(Rb,3,3)
    #input_data['Rb']= div(Rb,3,3)
    forecast['Rb'] = Rb

    # Afの計算
    forecast['Af'] = _Gb/_Go  # _G2

    Af = forecast['Af'].values

    # fの計算
    forecast['f'] = np.sqrt(_Gb/_G)  # _G2
    # 欠損値を0で置き換える
    forecast = forecast.fillna(0)
    f = forecast['f'].values

    def sin3(x):
        return math.sin(x)*math.sin(x)*math.sin(x)

    # _Gtの計算#２の３乗#累乗np.power(2, 3) np.power(np.sin(beta/2),3) or def 3sin(x)
    _Gt = (_Gb+_Gd*Af)*Rb\
        + (_Gd*(1-Af)*((1+math.cos(beta))/2)
           * np.abs(1+f*sin3(beta/2)))\
        + (_G*rho_g*((1-math.cos(beta))/2))
    _Gt[np.isnan(_Gt)] = 0

    forecast['_Gt'] = _Gt

    # Tc(the PV cell temperature [°C])の計算
    forecast['temperature[K]'] = forecast['temperature'] + 273.15
    Ta = forecast['temperature[K]'].values

    #input_data['Tc(the PV cell temperature)[°C]']\
    # = Ta+_Gt*(tau_a/UL)*(1-eta_c/tau_a)

    forecast['Tc(the PV cell temperature)[K]']\
        = Ta+_Gt*((Tc_NOCT-Ta_NOCT)/Gt_NOCT)*(1-eta_c/tau_a)

    Tc = forecast['Tc(the PV cell temperature)[K]'].values

    forecast['Tc(the PV cell temperature)[°C]'] = forecast['Tc(the PV cell temperature)[K]']-273.15

    # Ppv(the output of the PV array)[kW]の計算   #発散の処理div(x)を使用
    #input_data['Ppv(the output of the PV array)[Wh]']\
    # =div(YPV*fpv*(_Gt/_Gt_STC)*np.abs((1+ap*(Tc-Tc_STC))), YPV,YPV)

    # Ppv(the output of the PV array)[kW]の計算   #発散の処理div(x)を使用2
    forecast['Ppv(the output of the PV array)[Wh]']\
        = 1.28296*YPV*fpv*(_Gt/_Gt_STC)*np.abs((1+ap*(Tc-Tc_STC)))

    Ppv = forecast['Ppv(the output of the PV array)[Wh]'].values

    #input_data['Left'] = YPV*fpv*(_Gt/_Gt_STC)
    #input_data['Right'] =np.abs((1+ap*(Tc-Tc_STC)))

    # 入力初期値をデータフレームにする。
    input_parameter_name = ['Gsc', 'Gt_NOCT', 'Ta_NOCT', 'Tc_NOCT', 'Tc_STC', 'UL',
                            'YPV', 'ap', 'beta', 'eta_c', 'fpv', 'ganma', 'lam', 'phi',
                            'rho_g', 'tau_a', 'total_cost', 'Op_year', 'cost']

    input_parameter_list = [Gsc, Gt_NOCT, Ta_NOCT, Tc_NOCT, Tc_STC, UL,
                            YPV, ap, beta, eta_c, fpv, ganma, lam, phi,
                            rho_g, tau_a, total_cost, Op_year, cost]

    def float2DataFrame(name, list):
        df0 = pd.DataFrame({str(name[0]): [list[0]]})
        for i in range(len(list)-1):
            a = float(list[i+1])
            df = pd.DataFrame({str(name[i+1]): [a]})
            df0 = pd.concat([df0, df], axis=1)
        return df0

    input_parameter = float2DataFrame(
        input_parameter_name, input_parameter_list)
    input_parameter

    # Timeをインデックスに設定ものをdfと定義
    #df = input_data.set_index("Time")
    input_parameter.to_csv(
        'PV_input_parameter_result.csv', encoding="SHIFT-JIS")

    df = forecast.set_index('DateTime')

    forecast_edit = forecast.loc[:, ['Date', 'temperature', 'radiation', 'MSL', 'wind 10m[m/s]', 'wind output[kW]', 'wind output from power curve[kW]', 'Tc(the PV cell temperature)[°C]',
                                     'Ppv(the output of the PV array)[Wh]']]

    # データの書き出し
    filename = file_dir_path+'/forecast'+year+month+day+time+'.csv'
    forecast.to_csv(filename, encoding="SHIFT-JIS")

    # データの書き出し
    filename = file_dir_path+'/forecast_edit'+year+month+day+time+'.csv'
    forecast_edit.to_csv(filename, encoding="SHIFT-JIS")

    endday = int(endday)
    day = int(day)
    day2 = int(day2)
    day3 = int(day3)

    if day == endday:
        print('break!')
        break
    else:
        day = day+1
        day2 = day2+1
        day3 = day3+1

    day = str(day)
    day2 = str(day2)
    day3 = str(day3)
    endday = str(endday)

OK
52
OK


/home/yuichiro/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:298: RuntimeWarning: invalid value encountered in true_divide
/home/yuichiro/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:304: RuntimeWarning: invalid value encountered in greater_equal
/home/yuichiro/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:305: RuntimeWarning: invalid value encountered in less
/home/yuichiro/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:342: RuntimeWarning: invalid value encountered in true_divide


52
OK
52
OK
52
OK
52
OK
52
OK
52
OK
52


In [5]:
forecast.columns

Index(['DateTime', 'Date', 'Year', 'Month', 'Day', 'Hour', 'wind 10m[m/s]',
       'index', 'temperature', 'index', 'radiation', 'index', 'Uwind', 'index',
       'Vwind', 'index', 'MSL', 'wind output STC[kW]', 'rho',
       'wind output[kW]', 'wind output from power curve[kW]', 'ts', 'ω', 'n',
       'δ the solar declination', 'cosθ', 'θ(radian)', 'θ(degree)', 'cosθz',
       'θz(radian)', 'θz(degree)',
       'Gon(the extraterrestrial normal radiation)[kW/m2]',
       'Go(the extraterrestrial horizontal radiation)[kW/m2]',
       '_Go(the extraterrestrial horizontal radiation averaged over the time step )[kW/m2]',
       'the global horizontal radiation[kW/m2]', 'kt(the clearness index)',
       '_Gd(the diffuse radiation)[kW/m2]', '_Gb(the beam radiation)[kW/m2]',
       '_G', 'Rb', 'Af', 'f', '_Gt', 'temperature[K]',
       'Tc(the PV cell temperature)[K]', 'Tc(the PV cell temperature)[°C]',
       'Ppv(the output of the PV array)[Wh]'],
      dtype='object')

In [6]:
forecast.loc[:,['Date', 'temperature', 'radiation','MSL', 'wind 10m[m/s]', 'wind output[kW]','wind output from power curve[kW]', 'Tc(the PV cell temperature)[°C]',
           'Ppv(the output of the PV array)[Wh]']]

,Date,temperature,radiation,MSL,wind 10m[m/s],wind output[kW],wind output from power curve[kW],Tc(the PV cell temperature)[°C],Ppv(the output of the PV array)[Wh]
0,2019-06-23 10:00:00,20.719351,648.550238,1007.005981,3.978178,0.016825,0.008634,34.826071,116.594875
1,2019-06-23 11:00:00,20.741964,759.668122,1007.086975,4.890138,0.032511,0.019299,37.611026,138.398315
2,2019-06-23 12:00:00,20.829550,817.506268,1007.066406,5.646059,0.045498,0.030916,39.110079,149.378823
3,2019-06-23 13:00:00,20.805898,744.073063,1007.041443,5.832871,0.048713,0.034175,37.329236,135.663773
4,2019-06-23 14:00:00,20.745748,684.106339,1006.773499,5.986942,0.051359,0.036978,35.596800,122.496495
5,2019-06-23 15:00:00,20.804189,609.448936,1006.519409,5.781126,0.047799,0.033257,33.447894,104.884002
6,2019-06-23 16:00:00,20.906973,399.432730,1006.320557,5.334172,0.040095,0.025818,28.493277,63.753444
7,2019-06-23 17:00:00,20.920858,320.840704,1006.188721,5.457079,0.042199,0.027776,25.821820,41.472939
8,2019-06-23 18:00:00,20.919760,246.124532,1006.033997,6.034533,0.052108,0.037865,21.940685,8.725974
9,2019-06-23 19:00:00,20.867635,99.750000,1006.430725,5.709994,0.046562,0.032014,23.759761,24.604230


In [7]:
Pwt_values.shape

(52, 1)